In [3]:
import numpy as np
import pycosat
import itertools
import random

In [4]:
# this cell is responsible for generating cnf files that represent circledoku's

def encode_doku_as_cnf(doku, show):
    n = len(doku)
    names = initialize(n)
    cnf = []
    cnf = encode_exactly_one(cnf, names, show)
    cnf = encode_ring(cnf,names,show)
    cnf = encode_wedge(cnf,names,show)
    cnf = encode_slice(cnf,names)
    cnf.extend(encode_givens(names,doku,show))
    if show:
        print("the doku is now translated to propositional logic in cnf format")
    return cnf

def initialize(n):
    names = np.zeros([n,2*n,2*n],dtype=np.int)
    for i in range(n):
        for j in range(2*n):
            for k in range(2*n):
                names[i][j][k]= i*(2*n)**2 + j*2*n + k+1
    return names.tolist()

def encode_exactly_one(cnf, names, show):
    if show:
        print("enter function encode_exactly_one")
    cnf.extend(encode_at_most_one(cnf, names, show))
    cnf.extend(encode_at_least_one(cnf, names, show))
    return cnf

def encode_at_most_one(cnf, names, show):
    if show:
        print("enter function encode_at_most_one")
    n = len(names)
    #enc = []
    for i in range(n):
        for j in range(2*n):
            if show:
                print("in row ", i, ", column", j, ", there may only be at most 1 number, hence the following clauses:")
            for k in range(2*n):
                for l in range(1,(2*n)-k):
                    arr = [-1*names[i][j][k],-1*names[i][j][k+l]]
                    if show:
                        print(arr)
                    per_entry = cnf.insert(0,arr)
    return cnf

def encode_at_least_one(cnf,names,show):
    if show:
        print("enter function encode_at_least_one")
        print("for every field we expect a clause of", len(names[0]), "positive literals:")
    for i in names:
        atleast=[]
        for j in i:
            atleast.append(j)
            if show:
                print(j)
        cnf.extend(atleast)
    return cnf

def encode_ring(cnf, names, show):
    if show:
        print("")
        print("enter encode_ring function")
        print("")
    n = len(names)
    for i in range(n):
        for k in range(2*n):
            at_least_clause = []
            if show:
                print("in row ", i, ", there may not be more than one", k+1)
            for j in range(2*n): 
                at_least_clause.append(names[i][j][k])
                at_most_clause = []
                for l in range(1,(2*n)-j):
                    at_most_clause = [-1*names[i][j][k], -1*names[i][j+l][k]]
                    cnf.extend([at_most_clause])
                    if show:
                        print(at_most_clause)
            cnf.extend([at_least_clause])
            if show:
                print("but there must be at least one ", k+1)
                print(at_least_clause)                
    return cnf

def encode_wedge(cnf, names, show):
    if show:
        print("")
        print("enter encode_wedge function")
        print("")
    n = len(names) # n is number of rings, but also number of wedges
    # first, collect all the variables that stand for a number "1" (or"k") in a wedge
    # then, construct clauses that say one of these variables must be true
    # but no more than one of thesevariables may be true
    for w in range(n): # per wedge
        for k in range(2*n): # per number
            wedge_content = []
            for i in range(n): # per row
                for j in range(2*w, 2*w+2): # per column
                    wedge_content.append(names[i][j][k])
            cnf.extend([wedge_content])
            if show:
                print("in wedge ", w, "there must be the number", k+1)
                print(wedge_content)
                print("but only one ", k+1)
            for l in range(2*n): # loop over the wedgecontent
                for m in range(1, 2*n - l):
                    clause = [-1*wedge_content[l], -1*wedge_content[l+m]]
                    if show:
                        print(clause)
    return cnf

def encode_slice(cnf, names):
    n = len(names)
    for i in range(n):
        sliced = getslice(i,names)
        atleast_n =[]
        comb_n = []
        for l in range(2*n):
            atleast= [k[l] for k in sliced]
            atleast_n.append(atleast)
            atleast_neg = [-x for x in atleast]
            comb = [list(z) for z in itertools.combinations(atleast_neg,n)]
            atleast_n.extend(comb)
        cnf.extend(atleast_n)
    return cnf

def getslice(k,names):
    slice_C = []
    n = len(names)
    for i in range(n):
        for j in range(k,2*n,n):
            a = names[i][j]
            slice_C.append(a)
    return slice_C

def encode_givens( names, puzzle, show):
    givens = []
    n = len(names)
    for i in range(n):
        for j in range(2*n):
            for k in range(2*n):
                if puzzle[i,j] == k+1:
                    clause = [names[i][j][k]]
                    givens.extend([clause])
                    if show:
                        print("givens-clause: ", clause)
    return givens  

In [13]:
# this cell can be used to test the encoding of a sudoku
a = np.loadtxt('valid_circledoku_2_rings_#0', delimiter=',', dtype='int' )
a[0,0]=0
cnf = encode_doku_as_cnf(a, False)
print(cnf)
pycosat.solve(cnf)

[[-31, -32], [-30, -32], [-30, -31], [-29, -32], [-29, -31], [-29, -30], [-27, -28], [-26, -28], [-26, -27], [-25, -28], [-25, -27], [-25, -26], [-23, -24], [-22, -24], [-22, -23], [-21, -24], [-21, -23], [-21, -22], [-19, -20], [-18, -20], [-18, -19], [-17, -20], [-17, -19], [-17, -18], [-15, -16], [-14, -16], [-14, -15], [-13, -16], [-13, -15], [-13, -14], [-11, -12], [-10, -12], [-10, -11], [-9, -12], [-9, -11], [-9, -10], [-7, -8], [-6, -8], [-6, -7], [-5, -8], [-5, -7], [-5, -6], [-3, -4], [-2, -4], [-2, -3], [-1, -4], [-1, -3], [-1, -2], [-31, -32], [-30, -32], [-30, -31], [-29, -32], [-29, -31], [-29, -30], [-27, -28], [-26, -28], [-26, -27], [-25, -28], [-25, -27], [-25, -26], [-23, -24], [-22, -24], [-22, -23], [-21, -24], [-21, -23], [-21, -22], [-19, -20], [-18, -20], [-18, -19], [-17, -20], [-17, -19], [-17, -18], [-15, -16], [-14, -16], [-14, -15], [-13, -16], [-13, -15], [-13, -14], [-11, -12], [-10, -12], [-10, -11], [-9, -12], [-9, -11], [-9, -10], [-7, -8], [-6, -8], [

[-1,
 2,
 -3,
 -4,
 5,
 -6,
 -7,
 -8,
 -9,
 -10,
 11,
 -12,
 -13,
 -14,
 -15,
 16,
 -17,
 -18,
 -19,
 20,
 -21,
 -22,
 23,
 -24,
 25,
 -26,
 -27,
 -28,
 -29,
 30,
 -31,
 -32]

In [9]:
# this cell is responsible for taking a full valid circledoku
# and stripping it down to a proper one with as few givens as it can manage
# note that this does not mean that it generates circledoku's with the absolute minimum of givens

def reduce_CircleDoku(doku_puzzle):
    # this function takes a full, valid circledoku as input
    # and gives a proper circledoku with as few given as possible as output
    # the encoding for the rules of a circledoku are general and the same for every instance
    # only the givens change. We make use of that fact
    n = len(doku_puzzle)
    basis = np.zeros(([n, 2*n]))
    names = initialize(n)
    cnf_base = encode_doku_as_cnf(basis, False) # the basic rules that hold for all circledoku's of size n, in cnf format
    # below we shall transform the puzzle to a list of indices and their content
    # [[1,2],[3,4]] becomes [[0,0,1],[0,1,2],[1,0,3],[1,1,4]]
    doku_indices = []
    for i in range(n):
        for j in range(2*n):
            doku_indices.append([i,j,doku_puzzle[i,j]])
    random.shuffle(doku_indices, random.random)
    for i in range( len(doku_indices) ):
        save = doku_indices[i][2]
        doku_indices[i][2] = 0 # what if we removed this number from the puzzle? would it still be proper?
        puzzle = indices_to_puzzle(doku_indices)
        cnf = []
        cnf.extend(cnf_base)
        givens = encode_givens(names, puzzle, False)
        cnf.extend(givens)
        if len(list(pycosat.itersolve(cnf))) > 1: # if there is more than one solution, the puzzle is improper 
            doku_indices[i][2] = save # if the puzzle has become improper, restore the number, else, leave it empty
    puzzle = indices_to_puzzle(doku_indices)
    return puzzle
    
def indices_to_puzzle(doku_indices):
    n = int(len(doku_indices)/4)
    doku_puzzle = np.zeros((n, 2*n), dtype=np.int)
    doku_puzzle.tolist()
    for i in range(len(doku_indices)):
        x = doku_indices[i][0]
        y = doku_indices[i][1]
        doku_puzzle[x][y] = doku_indices[i][2]
    return doku_puzzle

In [10]:
# with this cell we can test the reduction of full circledoku's to fun puzzles
a = np.loadtxt('valid_circledoku_2_rings_#0', delimiter=',', dtype='int' )
reduce_CircleDoku(a)

array([[2, 0, 0, 0],
       [4, 3, 0, 0]])